# Ranking My Friends on Generals

- During the course of the pandemic, I started playing a decade old game called Command&Conquer Generals:Zero Hour with my brother and a bunch of friends. 

- Out of curiousity, we started keeping track of the stats from all the games and show wins and losses for each person. 

- This is a team game though so being able to pull out an individual's overall excellence and measure of how much better the person was a little more tricky. 

- For this project, I wanted to ingest the stats, create a model for predicting each person's relative strength, and end up with a score that could be used to create more even teams for future games. 

The scores are stored in a google sheet located [here](https://docs.google.com/spreadsheets/d/1ks6mqMbTgVFkQE-rZDByKVnGH4WMRMOdLSdabeMfZaA)

In [1]:
###
#---
#Created on Mon Feb  8 09:36:57 2021

#---
###

#import all the libraries we'll use
import math
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean, std

import matplotlib.pyplot as plt
from itertools import combinations

import pickle
import os.path

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

RANDOM_STATE = 42 
MAX_NUMBER_OF_GAMES = 25

The games were stored on a google sheet so the following functions ingest the most up-to-date data. 

In [ ]:
def gsheet_api_check(SCOPES):
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    result = sheet.values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=DATA_TO_PULL).execute()
    values = result.get('values', [])
    
    if not values:
        print('No data found.')
    else:
        rows = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                  range=DATA_TO_PULL).execute()
        data = rows.get('values')
        print("COMPLETE: Data copied")
        return data
    
def push_sheet_data(SCOPES,SPREADSHEET_ID,RANGE, DATA_TO_PUSH):
    creds = gsheet_api_check(SCOPES)
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    body = {
        'values': DATA_TO_PUSH
    }
    result = sheet.values().update(
        spreadsheetId=SPREADSHEET_ID, range=RANGE,
        valueInputOption='USER_ENTERED', body=body).execute()
    data = result.get('updatedCells')
    print('{0} cells updated.'.format(data))
    
    return data    

## Import Data from Google Sheets

In [16]:
    
# If modifying these scopes, delete the file token.pickle.
#SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.
SPREADSHEET_ID = '1ks6mqMbTgVFkQE-rZDByKVnGH4WMRMOdLSdabeMfZaA'

#Pulls data from the entire spreadsheet tab.
#DATA_TO_PULL = 'Games'
#or
#Pulls data only from the specified range of cells.
DATA_TO_PULL = 'Games!A1:Q4000'
data = pull_sheet_data(SCOPES,SPREADSHEET_ID,DATA_TO_PULL)
games = pd.DataFrame(data[1:], columns=data[0])
games = games.set_index('Index',drop=True)
#df.head()

numeric_columns = ['Team', 'Win', 'Game', 'Units Created',
       'Units Lost', 'Units Destroyed', 'Buildings Constructed',
       'Buildings Lost', 'Buildings Destroyed', 'Supplies Collected', 'Rank',
       'Inverse Rank', 'Normalized Rank']

for col in numeric_columns:
    games[col] = pd.to_numeric(games[col]).copy()
games['Date'] = pd.to_datetime(games['Date'])
games.head()

COMPLETE: Data copied


,Date,Name,Faction,Team,Win,Game,Units Created,Units Lost,Units Destroyed,Buildings Constructed,Buildings Lost,Buildings Destroyed,Supplies Collected,Rank,Inverse Rank,Normalized Rank
Index,,,,,,,,,,,,,,,,
1,2021-02-03,Matt,USA,1.0,1.0,1,216.0,195.0,140.0,47.0,6.0,14.0,360860.0,1,7,0.125
2,2021-02-03,Skippy,USA,2.0,NaN,1,129.0,132.0,115.0,63.0,12.0,9.0,260440.0,2,6,0.250
3,2021-02-03,Neo,China,1.0,1.0,1,175.0,83.0,83.0,53.0,12.0,8.0,233450.0,3,5,0.375
4,2021-02-03,TVH,USA,1.0,1.0,1,98.0,93.0,93.0,47.0,11.0,8.0,211565.0,4,4,0.500
5,2021-02-03,Pancake,China,2.0,NaN,1,122.0,64.0,137.0,40.0,11.0,6.0,192521.0,5,3,0.625


## Basic Cleaning

There may be some rows of data that need cleaning. 
The main method of dealing with it is to eliminate the problem games since we have enough data regardless. 


In [17]:

#Remove any row that doesn't have an index
games = games[(~games.index.isna())&(~games.Game.isna())].copy()

#Set the Game row to be integers instead of floats since we'll use it to make ranges
games.loc[:,'Game']=games.Game.astype(np.int32)

#Remove any game where team data isn't present and only include the columns up to Normalized Rank
games=games.loc[~games.Team.isna(),games.columns[0:16]].copy()

#Win's are designated with a 1 if there's a win, and are empty (NA) if it's a loss
#If it's a loss, we need to use a 0, otherwise it will throw off our average win calculations
games.loc[games.Win.isna(),'Win'] = games.loc[games.Win.isna(),'Win'].fillna(0)

#For every person calculate Win ratio and average rank from normalized rankings
for name in games.Name.unique():
    #print(name)
    games.loc[games.Name==name,'Win Ratio'] = games.loc[games.Name==name,'Win'].mean()
    games.loc[games.Name==name,'Avg Rank'] = games.loc[games.Name==name,'Normalized Rank'].mean()

#We use team 1 to designate which team won in another program to see which people win and lose the most together,
#but we need to mix this up or the computer's predictive model would take that as way to easily cheat.

for i in range( int(games.Game.max()+1)):
    #randomize the team numbers for each game
    team_1 = np.random.choice([0,1])
    team_2 = 1-team_1
    games.loc[(games.Game==i)&(games.Team==1),'Team'] = team_1
    games.loc[(games.Game==i)&(games.Team==2),'Team'] = team_2
    
games.head()    

,Date,Name,Faction,Team,Win,Game,Units Created,Units Lost,Units Destroyed,Buildings Constructed,Buildings Lost,Buildings Destroyed,Supplies Collected,Rank,Inverse Rank,Normalized Rank,Win Ratio,Avg Rank
Index,,,,,,,,,,,,,,,,,,
1,2021-02-03,Matt,USA,0.0,1.0,1,216.0,195.0,140.0,47.0,6.0,14.0,360860.0,1,7,0.125,0.588889,0.475370
2,2021-02-03,Skippy,USA,1.0,0.0,1,129.0,132.0,115.0,63.0,12.0,9.0,260440.0,2,6,0.250,0.456432,0.728438
3,2021-02-03,Neo,China,0.0,1.0,1,175.0,83.0,83.0,53.0,12.0,8.0,233450.0,3,5,0.375,0.513636,0.441088
4,2021-02-03,TVH,USA,0.0,1.0,1,98.0,93.0,93.0,47.0,11.0,8.0,211565.0,4,4,0.500,0.439024,0.708537
5,2021-02-03,Pancake,China,1.0,0.0,1,122.0,64.0,137.0,40.0,11.0,6.0,192521.0,5,3,0.625,0.455782,0.620408


## Create New Training Data

We're now going to create synthetic training data. 
For this, we'll look at each player's statistics, figure out for this particular game, what his stats were for the last N games, take the average, and use that as their nominal stats each game, then pretend the two teams played each other and make the logistic regression model predict which team will win. 

In [18]:
#Create a new column that contains the winning team
df =  pd.DataFrame(games.loc[games.Win==1,:].groupby('Game').mean().Team.astype(np.int32))
df.columns = ['Winning_Team']

#We'll use this for segmenting out which columns to use for predicting the winning team
prediction_columns = ['Units Created', 
                      'Units Lost', 
                      'Units Destroyed',
                      'Buildings Constructed',
                      'Buildings Lost', 
                      'Buildings Destroyed', 
                      'Supplies Collected', 
                      'Avg Rank', 
                      'Win Ratio']

games_copy = games.copy()
df.head()

,Winning_Team
Game,
1,0
2,1
3,0
4,1
5,1


In [19]:
#for each player, and each game, create their average win and rank stats for the previous N games
for i in range( int(games_copy.Game.max()+1)):
    for name in games_copy.loc[games_copy.Game==i,'Name'].unique():
        name_bool=games_copy.Name==name
        game_bool=games_copy.Game==i
        games_copy.loc[(name_bool)&(game_bool),'Win Ratio'] = games_copy.loc[(name_bool)&(games_copy.Game<=i),'Win'].tail(MAX_NUMBER_OF_GAMES).mean()
        games_copy.loc[(name_bool)&(game_bool),'Avg Rank'] = games_copy.loc[(name_bool)&(games_copy.Game<=i),'Normalized Rank'].tail(MAX_NUMBER_OF_GAMES).mean()
games_copy.head()    

,Date,Name,Faction,Team,Win,Game,Units Created,Units Lost,Units Destroyed,Buildings Constructed,Buildings Lost,Buildings Destroyed,Supplies Collected,Rank,Inverse Rank,Normalized Rank,Win Ratio,Avg Rank
Index,,,,,,,,,,,,,,,,,,
1,2021-02-03,Matt,USA,0.0,1.0,1,216.0,195.0,140.0,47.0,6.0,14.0,360860.0,1,7,0.125,1.0,0.125
2,2021-02-03,Skippy,USA,1.0,0.0,1,129.0,132.0,115.0,63.0,12.0,9.0,260440.0,2,6,0.250,0.0,0.250
3,2021-02-03,Neo,China,0.0,1.0,1,175.0,83.0,83.0,53.0,12.0,8.0,233450.0,3,5,0.375,1.0,0.375
4,2021-02-03,TVH,USA,0.0,1.0,1,98.0,93.0,93.0,47.0,11.0,8.0,211565.0,4,4,0.500,1.0,0.500
5,2021-02-03,Pancake,China,1.0,0.0,1,122.0,64.0,137.0,40.0,11.0,6.0,192521.0,5,3,0.625,0.0,0.625


## Reduce each game to a single row of stats
- For predicting each game, we're going to sum the stats for each team, then take the difference. 
- For games with large negative numbers, this will indicate that team 0 won, for mostly positive, it would show team 1 won.
- For predicting each game, most of the stats should be added except for the game, team, and win stats. 

In [21]:

values = games_copy.loc[games_copy.Team==1,:].iloc[:,3:].groupby('Game').agg({'Team':'mean', 
                                                           'Win':'mean', 
                                                           'Game':'mean', 
                                                           'Units Created':'sum', 
                                                           'Units Lost':'sum', 
                                                           'Units Destroyed':'sum',
                                                           'Buildings Constructed':'sum', 
                                                           'Buildings Lost':'sum', 
                                                           'Buildings Destroyed':'sum',
                                                           'Supplies Collected':'sum', 
                                                           'Rank':'sum', 
                                                           'Inverse Rank':'sum', #not used
                                                           'Normalized Rank':'sum', #not used
                                                           'Win Ratio':'sum', 
                                                           'Avg Rank':'sum' 
                                                           }) - games_copy.loc[
                                                               games_copy.Team==0,:].iloc[:,3:].groupby('Game').agg({'Team':'mean', 
                                                           'Win':'mean', 
                                                           'Game':'mean', 
                                                           'Units Created':'sum', 
                                                           'Units Lost':'sum', 
                                                           'Units Destroyed':'sum',
                                                           'Buildings Constructed':'sum', 
                                                           'Buildings Lost':'sum', 
                                                           'Buildings Destroyed':'sum',
                                                           'Supplies Collected':'sum', 
                                                           'Rank':'sum', 
                                                           'Inverse Rank':'sum', #not used
                                                           'Normalized Rank':'sum', #not used
                                                           'Win Ratio':'sum', 
                                                           'Avg Rank':'sum', 
                                                           })

#create the difference columns
diff_cols = []
for col in prediction_columns:
    column_name = col+'_diff'
    diff_cols += [column_name]
    #this really isn't needed anymore but the winning team will still be needed for training value
    df.loc[:,column_name] = values.loc[:,col]
    
#create nominal game stats based on median stats for each player    
predicted_games = []    

#make stats for each game
for game in games_copy.Game.unique():  
    #print('game ', game)
    team_values = []
    
    #make stats for each team
    for team in range(2):
        #print('team ', team)
        names = []
        #make stats for each player on this team
        games_copy.loc[(games_copy.Name==name)&(games.Game<=i),'Win'].tail(MAX_NUMBER_OF_GAMES).mean()
        for name in games_copy.loc[(games_copy.Game==game)&(games_copy.Team==team),'Name'].values:
            name_stats = games_copy.loc[(games_copy.Name==name)&(games.Game<=game)].tail(MAX_NUMBER_OF_GAMES).iloc[:,6:].median()
            name_stats['Win Ratio'] = games_copy.loc[(games_copy.Name==name)&(games_copy.Game<=game),'Win'].tail(MAX_NUMBER_OF_GAMES).mean()
            name_stats['Avg Rank'] = games_copy.loc[(games_copy.Name==name)&(games_copy.Game<=game),'Normalized Rank'].tail(MAX_NUMBER_OF_GAMES).mean()
            names += [name_stats]
            names[-1].loc['Win_avg'] = games_copy.loc[(games_copy.Name==name)&
                                                      (~games_copy.Team.isna())&
                                                      (games_copy.Game<=game)].Win.fillna(0).tail(MAX_NUMBER_OF_GAMES).mean()
            #print(name)
        #combine all the medians and sum them together
        #Summing works better than an average or median since if the teams have uneven number of players, the weight is on the side with more players

        team_values  += [pd.concat(names, axis = 1).T.sum()]
    predicted_games +=[team_values[1]-team_values[0]]
X_generated = pd.concat(predicted_games, axis = 1).T[prediction_columns]
X_generated.columns = diff_cols
X_generated.head()

,Units Created_diff,Units Lost_diff,Units Destroyed_diff,Buildings Constructed_diff,Buildings Lost_diff,Buildings Destroyed_diff,Supplies Collected_diff,Avg Rank_diff,Win Ratio_diff
0,-90.0,-79.0,75.0,4.0,0.0,-1.0,-99050.0,0.500000,-4.000000
1,199.0,251.5,65.0,57.5,-13.0,45.0,378716.0,-0.229167,2.500000
2,-19.0,-50.0,96.5,-64.5,31.5,-25.5,-81672.5,0.638889,-2.166667
3,208.0,202.0,15.0,54.0,58.5,-37.0,173173.5,0.173611,1.083333
4,95.5,162.5,56.5,44.5,22.5,12.0,350781.5,-1.067361,1.633333


## Prediction Test

Create a logistic regression model, with test and training splits, and 10 cross validation folds for determing accuracy. 


In [ ]:
#predict     
#from sklearn.model_selection import GridSearchCV

y_cols= ['Winning_Team']
X = X_generated
y = np.ravel(df[y_cols])

#standard test, train split should be 20-30% held back for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#cross validation
cv = KFold(n_splits = 10, random_state=10, shuffle=True)


#solvers = ['newton-cg', 'lbfgs','liblinear', 'sag', 'saga']
#Cs = [1, 3, 10, 30, 100]
#parameters = {'logisticregression__solver':solvers, 'logisticregression__C':Cs}

#create model
#model = LogisticRegression(solver='lbfgs')
scaler = StandardScaler()
logreg = LogisticRegression(random_state=RANDOM_STATE)


#X_train = df[X_cols]
#y_train = np.ravel(df[y_cols])
#y_test = np.ravel(df[y_cols])

#simple pipeline of normalizing all the stats then applying logistic regression
pipe = make_pipeline(scaler, logreg)

#clf = GridSearchCV(pipe, parameters, cv=10)

#clf.fit(X, y)
pipe.fit(X_train, y_train)  # apply scaling on training data
pipe.fit(X, y)
#sorted(clf.cv_results_.keys())

In [ ]:
#score it
scores = cross_val_score(pipe, X, y, scoring = 'accuracy', cv=cv, n_jobs = -1)
print('10-fold cross validation accuracy: %.3f (%.3f stdev)' % (mean(scores), std(scores)))

#score it
print("Accuracy Score is " + "{:.2%}".format(pipe.score(X_test, y_test)))

## Gridsearch
I used the below code block for testing a grid search but after finding my best values, I stuck with them.

In [ ]:
#%%Grid of gridsearch
# def make_heatmap(ax, gs, is_sh=False, make_cbar=False):
#     """Helper to make a heatmap."""
#     results = pd.DataFrame.from_dict(gs.cv_results_)
#     results['params_str'] = results.params.apply(str)
#     if is_sh:
#         # SH dataframe: get mean_test_score values for the highest iter
#         scores_matrix = results.sort_values('iter').pivot_table(
#                 index='param_logisticregression__solver', columns='param_logisticregression__C',
#                 values='mean_test_score', aggfunc='last'
#         )
#     else:
#         scores_matrix = results.pivot(index='param_logisticregression__solver', columns='param_logisticregression__C',
#                                       values='mean_test_score')

#     im = ax.imshow(scores_matrix)

#     ax.set_xticks(np.arange(len(Cs)))
#     ax.set_xticklabels([x for x in Cs])
#     ax.set_xlabel('C', fontsize=15)

#     ax.set_yticks(np.arange(len(solvers)))
#     ax.set_yticklabels([x for x in solvers])
#     ax.set_ylabel('solver', fontsize=15)

#     # Rotate the tick labels and set their alignment.
#     plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
#              rotation_mode="anchor")

#     if is_sh:
#         iterations = results.pivot_table(index='param_gamma',
#                                          columns='param_C', values='iter',
#                                          aggfunc='max').values
#         for i in range(len(solvers)):
#             for j in range(len(Cs)):
#                 ax.text(j, i, iterations[i, j],
#                         ha="center", va="center", color="w", fontsize=20)

#     if make_cbar:
#         fig.subplots_adjust(right=0.8)
#         cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
#         fig.colorbar(im, cax=cbar_ax)
#         cbar_ax.set_ylabel('mean_test_score', rotation=-90, va="bottom",
#                            fontsize=15)


# fig, axes = plt.subplots(ncols=1, sharey=True)
# ax2 = axes

# make_heatmap(ax2, clf, make_cbar=True)


# ax2.set_title('GridSearch', fontsize=15)

# plt.show()

In [ ]:
#%%    importance of each feature, magnitude matters more than sign
fig, ax = plt.subplots()
ax.bar(x = np.arange(len(pipe.steps[1][1].coef_[0])), height = pipe.steps[1][1].coef_[0])
ax.set_xticks(np.arange(len(prediction_columns)))
ax.set_xticklabels(prediction_columns)
plt.xticks(rotation=-90)
plt.title('Feature Importance')
plt.show()

In [ ]:
#%%

#chances of one team possibilities

first_team=['Shift'] 
second_team= ['Hard', 'Hard',]
test_team = [first_team, second_team]
predicted_games = []
team_values = []
for team in range(2):
    #print('team ', team)
    names = []
    for name in test_team[team]:
        if name not in games.Name.unique():
            print(name +' not found')
            break
        name_stats = games_copy.loc[(games_copy.Name==name)].tail(MAX_NUMBER_OF_GAMES).iloc[:,6:].median()
        name_stats['Win Ratio'] = games_copy.loc[(games_copy.Name==name)&(games_copy.Game<=game),'Win'].tail(MAX_NUMBER_OF_GAMES).mean()
        name_stats['Avg Rank'] = games_copy.loc[(games_copy.Name==name)&(games_copy.Game<=game),'Normalized Rank'].tail(MAX_NUMBER_OF_GAMES).mean()
        names += [name_stats]
        names[-1].loc['Win_avg'] = games_copy.loc[(games_copy.Name==name)&
                                                  (~games_copy.Team.isna())&
                                                  (games_copy.Game<=game)].Win.fillna(0).tail(MAX_NUMBER_OF_GAMES).mean()

    team_values  += [pd.concat(names, axis = 1).T.sum()]
predicted_games +=[team_values[1]-team_values[0]]
X_predict = pd.concat(predicted_games, axis = 1).T[prediction_columns]
predicted_win = pipe.predict(X_predict)[0]
probability = pipe.predict_proba(X_predict)[0][predicted_win]
    #possibilities +=[{"Team 1":first_team, "Team 2":second_team, "Predicted Team": predicted_win+1, "Probability": probability}]

#print('\n\nBetween '+ ', '.join(first_team)+' and ' + ', '.join(second_team)+ ',\nI predict ' + ', '.join(test_team[predicted_win]) + ' with a '+"{:.2%}".format(probability)+' chance')

In [ ]:
#%%

#all possibilities
all_names = ['Neo', 'Shift','Matt', 'Spiff', 'Skippy', 'STM', 'Jack', 'Mike']
possibilities = []

non_parity_combos= int(len(list(combinations(all_names,int(len(all_names)/2))))/2)
for combo in list(combinations(all_names,int(len(all_names)/2)))[0:non_parity_combos]:
    first_team=list(combo)
    second_team= [x for x in all_names if x not in combo]
    test_team = [first_team, second_team]
    predicted_games = []
    team_values = []
    for team in range(2):
        #print('team ', team)
        names = []
        for name in test_team[team]:
            names += [games.loc[games.Name==name].iloc[:,6:].mean()]
            names[-1].loc['Win_avg'] = games.loc[(games.Name==name)&(~games.Team.isna())].Win.fillna(0).mean()
        team_values  += [pd.concat(names, axis = 1).T.mean()]
    predicted_games +=[team_values[1]-team_values[0]]
    X_predict = pd.concat(predicted_games, axis = 1).T[prediction_columns]
    predicted_win = pipe.predict(X_predict)[0]
    probability = pipe.predict_proba(X_predict)[0][predicted_win]
    possibilities +=[{"Team 1":first_team, "Team 2":second_team, "Predicted Team": predicted_win+1, "Probability": probability}]
all_runs = pd.DataFrame(possibilities)
#print('\n\nBetween '+ ', '.join(first_team)+' and ' + ', '.join(second_team)+ ',\nI predict ' + ', '.join(test_team[predicted_win]) + ' with a '+"{:.2%}".format(pipe.predict_proba(X_predict)[0][predicted_win])+' chance')
#print(all_runs.sort_values(by='Probability'))


first_team = all_runs.loc[all_runs.Probability ==all_runs.Probability.min(), 'Team 1'].values[0]
second_team = all_runs.loc[all_runs.Probability ==all_runs.Probability.min(), 'Team 2'].values[0]
predicted_win= all_runs.loc[all_runs.Probability ==all_runs.Probability.min(), 'Predicted Team'].values[0]-1

test_team = [first_team, second_team]
probability = all_runs.Probability.min()

print('\n\nFor '+ ', '.join(all_names)+',\nThe most even teams are '+ ', '.join(first_team)+' and ' + ', '.join(second_team)+ ',\nI predict ' + ', '.join(test_team[predicted_win]) + ' with a '+"{:.2%}".format(probability)+' chance')

In [ ]:
#%%
names = []
stats= {}
sheets_stats = [['Name', 'Predictive Rating']]
for name in games.Name.unique():
    only_team=[name]
    predicted_games = []
    team_values = []
    for team in range(2):
        #print('team ', team)
        names = []
        for name in only_team:
            name_stats = games_copy.loc[(games_copy.Name==name)].tail(MAX_NUMBER_OF_GAMES).iloc[:,6:].median()
            name_stats['Win Ratio'] = games_copy.loc[(games_copy.Name==name),'Win'].tail(MAX_NUMBER_OF_GAMES).mean()
            name_stats['Avg Rank'] = games_copy.loc[(games_copy.Name==name),'Normalized Rank'].tail(MAX_NUMBER_OF_GAMES).mean()
            names += [name_stats]
            names[-1].loc['Win_avg'] = games_copy.loc[(games_copy.Name==name)&
                                                      (~games_copy.Team.isna())].Win.fillna(0).tail(MAX_NUMBER_OF_GAMES).mean()
        team_values  += [pd.concat(names, axis = 1).T.sum()]
    predicted_games +=[team_values[0]]
    X_predict = pd.concat(predicted_games, axis = 1).T[prediction_columns]
    #predicted_win = pipe.predict(X_predict)[0]
    probability = pipe.predict_proba(X_predict)[0][1] #the player should always be 1
    #stats[name] = -math.log((1 - probability)/probability)
    stats[name] = probability
    #sheets_stats +=[[name, -math.log((1 - probability)/probability)]]
    sheets_stats +=[[name, probability]]
    games.loc[games.Name==name, 'Predictive Rating'] = stats[name]

RANGE = "'Team Maker'!P1:Q"+str(len(sheets_stats))
push_sheet_data(SCOPES,SPREADSHEET_ID,RANGE, sheets_stats)    

#make a quick plot to ensure easy isn't getting a good deal
games.groupby('Name')['Predictive Rating'].max().sort_values().plot.barh()
plt.title('Ranking based on last '+str(MAX_NUMBER_OF_GAMES)+' games')
plt.show()

In [ ]:
#%% update Tableau's Data spreadsheet and the google sheet's team maker info
games.to_excel('Generals Statistics.xlsx',sheet_name="Games")


v2 = []
v3 = []
v4 = []
for i in range(256):
    if i == 0:
        sheets_stats[0]+=['4v4']    
    number = bin(i+1).replace('0b',"")
    if (sum([int(x) for x in number])==4) & (len(number)==8):
        v4 +=[number]
        try:
            sheets_stats[len(v4)]+=[number]
        except:
            sheets_stats+=[["", "", number]]
for i in range(64):
    if i == 0:
        sheets_stats[0]+=['3v3']    
    number = bin(i+1).replace('0b',"")
    if (sum([int(x) for x in number])==3) & (len(number)==6):
        v3 +=[number]
        sheets_stats[len(v3)]+=[number]
for i in range(16):
    if i == 0:
        sheets_stats[0]+=['2v2']
    number = bin(i+1).replace('0b',"")
    if (sum([int(x) for x in number])==2) & (len(number)==4):
        v2 +=[number]
        sheets_stats[len(v2)]+=[number]

In [ ]:
        
#%%
from openpyxl import load_workbook
workbook = load_workbook(filename="Generals Statistics.xlsx")
workbook.create_sheet('Teams')
sheet = workbook['Teams']
for i, rows in enumerate(sheets_stats):
    sheet.cell(row=i+1, column=1).value = rows[0]
    sheet.cell(row=i+1, column=2).value = rows[1]
    try:
        sheet.cell(row=i+1, column=3).value = rows[2]
    except:
        pass
    try:
        sheet.cell(row=i+1, column=4).value = rows[3]
    except:
        pass
    try:
        sheet.cell(row=i+1, column=5).value = rows[4]
    except:
        pass
    
sheet.cell(row=len(games.Name.unique())+1, column=1).value = 'None'
sheet.cell(row=len(games.Name.unique())+1, column=2).value = 0
workbook.save(filename="Generals Statistics.xlsx")   

 